# Setup

In [1]:
from graphqlclient import GraphQLClient
import pandas as pd
import json
from pandas.io.json import json_normalize
from datetime import datetime

In [2]:
ENDPOINT = "https://api.thegraph.com/subgraphs/name/blocklytics/bancor"
client = GraphQLClient(ENDPOINT)

# QA

In [19]:
# Results must be paginated.
# Subgraphs return a maximum of 100 rows.
limit = 100
offset = 0
fetching_results = True
converters = []

# Fetch paginated results
while fetching_results:
    # This query manually removes certain converters
    # See https://blocklytics.org/blog/bancor-subgraph/
    QUERY = """
    {{
      converters(
        first:{0}, 
        skip:{1}, 
        where: {{
          id_not_in: ["0x77feb788c747a701eb65b8d3b522302aaf26b1e2", "0xcbc6a023eb975a1e2630223a7959988948e664f3", "0x11614c5f1eb215ecffe657da56d3dd12df395dc8", "0x2769eb86e3acdda921c4f36cfe6cad035d95d31b", "0x2ac0e433c3c9ad816db79852d6f933b0b117aefe", "0x37c88474b5d6c593bbd2e4ce16635c08f8215b1e", "0x445556b7215349b205997aaaf6c6dfa258eb029d", "0x46ffcdc6d8e6ed69f124d944bbfe0ac74f8fcf7f", "0x587044b74004e3d5ef2d453b7f8d198d9e4cb558"]
        }}
      ) {{
        id
        smartToken {{
          id
        }}
        tokenBalances {{
          token {{
            id
            symbol
            name
            decimals
          }}
          balance
        }}
      }}
    }}
    """.format(limit, offset)

    result = json.loads(client.execute(QUERY))
    converters += result['data']['converters']
    
    # Prepare for pagination
    result_length = len(result['data']['converters'])
    if limit == result_length:
        offset += limit
    else:
        fetching_results = False

# Load data into a new df
df = pd.DataFrame()

# Iterate over converters
i = 0
for converter in converters:
    # Skip empty converters
    if len(converter['tokenBalances']) == 0:
        continue
        
    converter_address = converter['id']
    smart_token_address = converter['smartToken']['id']
    
    df.at[i, 'exchange'] = converter_address
    
    # Iterate over token balances
    for tokenBalance in converter['tokenBalances']:
        token = tokenBalance['token']['id']
        
        # Skip converter's smart token
        # See https://blocklytics.org/blog/bancor-subgraph/
        if token == smart_token_address:
            continue

        # Handle remaining token details
        balance = tokenBalance['balance']
        symbol = tokenBalance['token']['symbol']
        name = tokenBalance['token']['name']
        decimals = tokenBalance['token']['decimals']
        
        try: # try/catch for missing token details
            balance_converted = float(balance) / 10 ** float(decimals)
        except:
            print("Could not find decimals for {0}. Assumed 18".format(token))
            balance_converted = float(balance) / 10 ** float(18)

        # Set base token to BNT or USDB
        
        if 'base' in df.columns and df.base.isna().iloc[i] == False:
            # Base has already been set for this converter
            df.at[i, 'token'] = token
            df.at[i, 'tokenSymbol'] = symbol
            df.at[i, 'tokenName'] = name
            df.at[i, 'tokenLiquidity'] = balance_converted
        else:
            # No base has been set for this converter
            if token == '0x1f573d6fb3f13d689ff844b4ce37794d79a7ff1c' or token == '0x309627af60f0926daa6041b8279484312f2bf060':
                # Bancor converts use BNT or USDB base
                df.at[i, 'base'] = token
                df.at[i, 'baseSymbol'] = symbol
#                 df.at[i, 'baseName'] = name
                df.at[i, 'baseLiquidity'] = balance_converted
            else:
                df.at[i, 'token'] = token
                df.at[i, 'tokenSymbol'] = symbol
#                 df.at[i, 'tokenName'] = name
                df.at[i, 'tokenLiquidity'] = balance_converted
                
    i += 1

df['basePrice'] = df['baseLiquidity'] / df['tokenLiquidity'] # Assumes 50% weight

print(df.shape)

Could not find decimals for 0xbdeb4b83251fb146687fa19d1c660f99411eefe3. Assumed 18
Could not find decimals for 0xe0b7927c4af23765cb51314a0e0521a9645f0e2a. Assumed 18
(169, 9)


## USDB Converters

In [26]:
df[df.baseSymbol == "USDB"][['baseSymbol', 'baseLiquidity', 'tokenSymbol', 'tokenLiquidity', 'basePrice']]\
    .sort_values(by='baseLiquidity', ascending=False)\
    .reset_index(drop=True)\
    .head(10)

,baseSymbol,baseLiquidity,tokenSymbol,tokenLiquidity,basePrice
0,USDB,14729.327688,ETH,101.391234,145.272200
1,USDB,10959.775516,DAI,10969.287803,0.999133
2,USDB,10934.936325,MET,25376.489130,0.430908
3,USDB,7461.649042,sUSD,7611.198503,0.980351
4,USDB,2563.690072,JRT,376416.104813,0.006811
5,USDB,214.768685,ZRX,1010.563314,0.212524
6,USDB,159.948703,USDT,155.465019,1.028840
7,USDB,144.531781,OMG,215.583761,0.670421
8,USDB,137.319888,REN,2901.627829,0.047325
9,USDB,135.825334,NEXO,1210.195736,0.112234


## BNT Converters

In [27]:
df[df.baseSymbol == "BNT"][['baseSymbol', 'baseLiquidity', 'tokenSymbol', 'tokenLiquidity', 'basePrice']]\
    .sort_values(by='baseLiquidity', ascending=False)\
    .reset_index(drop=True)\
    .head(10)

,baseSymbol,baseLiquidity,tokenSymbol,tokenLiquidity,basePrice
0,BNT,8.318618e+06,ETH,1.207134e+04,689.121261
1,BNT,4.034684e+05,BAT,4.636735e+05,0.870156
2,BNT,3.770589e+05,None,8.326363e+04,4.528494
3,BNT,3.428768e+05,ENJ,1.047348e+06,0.327376
4,BNT,1.991097e+05,CLN,8.008495e+06,0.024862
5,BNT,1.912496e+05,POE,2.488639e+07,0.007685
6,BNT,1.676712e+05,TKN,1.492371e+05,1.123522
7,BNT,1.670908e+05,EURS,3.334067e+04,5.011622
8,BNT,1.666551e+05,BNB,2.379603e+03,70.034826
9,BNT,1.660374e+05,None,7.439075e+01,2231.962952
